# Compute and analyze connected components in STRING networks

From [Wikipedia](https://en.wikipedia.org/wiki/Component_(graph_theory)):

> In graph theory, a component, sometimes called a connected component, of an undirected graph is a subgraph in which any two vertices are connected to each other by paths, and which is connected to no additional vertices in the supergraph.

See discussion in https://github.com/related-sciences/string-protein-network/issues/1

In [1]:
import pathlib

import pandas as pd
import scipy.sparse
import scipy.sparse.csgraph
import plotly.express as px

from utils import get_protein_info_df

In [2]:
protein_df = get_protein_info_df()
protein_df.head(2)

,index,protein_external_id,preferred_name,protein_size,annotation
0,0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...


In [3]:
matrix_paths = sorted(pathlib.Path("data/score-matrices").glob("*.sparse.npz"))
len(matrix_paths)

12

In [4]:
%%time
channel_to_components = {}
for path in matrix_paths:
    matrix = scipy.sparse.load_npz(path)
    channel = path.name.split(".")[0]
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.connected_components.html
    n_components, components = scipy.sparse.csgraph.connected_components(matrix, directed=False)
    components += 1  # one-index componenents for user-convenience
    channel_to_components[channel] = components

CPU times: user 3.65 s, sys: 1.24 s, total: 4.88 s
Wall time: 5.09 s


In [5]:
component_df = pd.DataFrame(
    channel_to_components,
    index=protein_df.set_index(["index", "protein_external_id", "preferred_name"]).index
)
component_df.head(2)

,,,coexpression,coexpression_transferred,combined_score,cooccurence,database,experiments,experiments_transferred,fusion,homology,neighborhood_transferred,textmining,textmining_transferred
index,protein_external_id,preferred_name,,,,,,,,,,,,
0,9606.ENSP00000000233,ARF5,1,1,1,1,1,1,1,1,1,1,1,1
1,9606.ENSP00000000412,M6PR,1,1,1,2,1,1,2,2,2,2,1,1


In [6]:
component_df.to_csv("data/clusters/connected-components.tsv.xz", sep="\t")

## Explore number of genes per component

In [7]:
# number of connected components per evidence channel
component_df.max(axis="rows")

coexpression                 2841
coexpression_transferred     1032
combined_score                213
cooccurence                 16895
database                     8711
experiments                  3128
experiments_transferred      4030
fusion                      13950
homology                     7316
neighborhood_transferred    15514
textmining                    517
textmining_transferred        633
dtype: int32

In [8]:
def get_count_df(channel: str) -> pd.DataFrame:
    """
    Compute number of genes per component and cumulative measures.
    """
    counts = component_df[channel].value_counts()
    df = counts.reset_index()
    df.columns = ["component", "n_genes"]
    df = df.sort_values(["n_genes", "component"], ascending=False)
    df["n_components"] = range(1, 1+len(df))
    df["n_genes_cumulative"] = df.n_genes.cumsum()
    df["percent_genes_cumulative"] = df["n_genes_cumulative"] / len(component_df)
    df["channel"] = channel
    return df

In [9]:
count_df = pd.concat([get_count_df(channel) for channel in component_df.columns])
count_df.head(2)

,component,n_genes,n_components,n_genes_cumulative,percent_genes_cumulative,channel
0,1,16681,1,16681,0.852550,coexpression
3,1883,4,2,16685,0.852755,coexpression


In [10]:
count_df.to_csv("data/clusters/connected-component-counts.tsv.xz", sep="\t", index=False, float_format="%.5g")

In [11]:
fig = px.line(
    count_df,
    x="n_components",
    y="percent_genes_cumulative",
    color='channel',
    hover_name="component",
    hover_data=["n_genes_cumulative", "n_genes"],
)
fig.update_traces(mode='lines+markers')
fig.update_xaxes(title="Number of connected components (sorted by n_genes)")
fig.update_yaxes(title="Cumulative proportion of genes in components", tickformat=".2%")
# load plotly.js JavaScript from an online CDN to reduce .ipynb file size
fig.show(renderer="notebook_connected")